# AI-Driven Algo Trading Strategies

# PART II – Live Prediction Using Trained LSTM Model

## Section 0: Import Libraries

In [2]:
# Core Python libraries
import numpy as np
import pandas as pd

# Visualization (optional)
import matplotlib.pyplot as plt

# Model and Scaler
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import tensorflow as tf
import joblib

# Live data
import yfinance as yf

# System / date handling (if needed for formatting/logging)
from datetime import datetime


## Section 1: Load the Trained Model and Scaler

In [3]:
# Define custom loss again (used during training)
def focal_loss_fixed(y_true, y_pred, gamma=2., alpha=0.25):
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
    return -K.mean(alpha * K.pow(1. - pt, gamma) * K.log(pt))

# Load model and scaler
model = load_model('lstm_model.h5', custom_objects={'focal_loss_fixed': focal_loss_fixed})
scaler = joblib.load('scaler.pkl')

## Section 2: Download Recent Stock Data (e.g., AAPL from yfinance)

In [4]:
df = yf.download('AAPL', period='100d', interval='1d')
df.reset_index(inplace=True)

C:\Users\BAISAMPAYAN\AppData\Local\Temp\ipykernel_29032\203224906.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', period='100d', interval='1d')
[*********************100%***********************]  1 of 1 completed


## Section 3: Feature Engineering (Same as Training)

In [5]:
df['High_Low_Range'] = df['High'] - df['Low']
df['Close_Change'] = df['Close'].pct_change()
df['SMA_10'] = df['Close'].rolling(10).mean()
df['SMA_50'] = df['Close'].rolling(50).mean()
df['Momentum_5'] = df['Close'] - df['Close'].shift(5)
df['Volatility_5'] = df['Close_Change'].rolling(5).std()
df.dropna(inplace=True)

features = ['Close', 'Volume', 'High_Low_Range', 'Momentum_5', 'Volatility_5', 'SMA_10', 'SMA_50']

## Section 4: Prepare Sequence and Predict

In [6]:
X_live = df[features].tail(15).values
X_scaled = scaler.transform(X_live)
X_input = X_scaled.reshape(1, 15, len(features))

y_pred = model.predict(X_input)
predicted_class = y_pred.argmax()
actions = ['Sell', 'Buy', 'Hold']
print("Predicted Action:", actions[predicted_class])
print("Prediction Probabilities:", y_pred)

c:\Users\BAISAMPAYAN\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Predicted Action: Hold
Prediction Probabilities: [[0.11376458 0.00494228 0.8812931 ]]


## Section 5: Mock Broker for Simulated Trade Execution

In [7]:
class MockBroker:
    def __init__(self, initial_cash=100000):
        self.cash = initial_cash
        self.position = 0
        self.trade_log = []

    def buy(self, price, date):
        if self.cash > price:
            self.position += 1
            self.cash -= price
            self.trade_log.append((date, 'BUY', price))

    def sell(self, price, date):
        if self.position > 0:
            self.position -= 1
            self.cash += price
            self.trade_log.append((date, 'SELL', price))

    def summary(self):
        print(f'Cash: ₹{self.cash:.2f}, Position: {self.position} shares')
        for log in self.trade_log:
            print(log)

## Section 6: Run Trade Based on Prediction

In [8]:
latest_date = df['Date'].iloc[-1]
latest_price = df['Close'].iloc[-1]

broker = MockBroker()
if predicted_class == 0:
    broker.sell(latest_price, latest_date)
elif predicted_class == 1:
    broker.buy(latest_price, latest_date)

broker.summary()

Cash: ₹100000.00, Position: 0 shares


## Conclusion:
---

### Part I Recap: Model Training & Evaluation
- We built and trained an LSTM model using historical stock data.
- Engineered technical indicators such as moving averages, momentum, and volatility.
- Applied label generation for Buy, Sell, and Hold signals.
- Handled class imbalance using focal loss.
- Evaluated model performance with classification metrics and visualizations.
- Simulated backtesting to compare model-driven returns with market performance.
- Exported the trained model and scaler for future inference.

---

### Part II Recap: Live Prediction & Trade Simulation
- Loaded the trained model and scaler.
- Fetched recent stock data using `yfinance`.
- Applied the same preprocessing and feature engineering as in training.
- Used the model to predict real-time trading actions.
- Simulated buy/sell/hold decisions using a mock trading broker.

---

Keep experimenting, stay curious, and remember: in trading, **discipline beats complexity**. Happy trading!
